In [1]:
import pandas as pd

# Paths to dataset
real_df = pd.read_csv(r"C:\Users\hiran\OneDrive\Desktop\AI 1-0\Datasets\BIGGER DATASET\FEATURES\real_audio_features.csv")
fake_df = pd.read_csv(r"C:\Users\hiran\OneDrive\Desktop\AI 1-0\Datasets\BIGGER DATASET\FEATURES\fake_audio_features.csv")

# 1 for fake, 0 for real
real_df['label'] = 0
fake_df['label'] = 1

# One dataframe for real and fake
combined_df = pd.concat([real_df, fake_df], ignore_index=True)

In [62]:
import numpy as np
import pandas as pd
import ast

# Summary statistics
def summarize_array_features(df):
    array_cols = []
    
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = df[col].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x)
                
                if df[col].apply(lambda x: isinstance(x, np.ndarray)).all():
                    array_cols.append(col)
                    
                    df[col + '_mean'] = df[col].apply(np.mean)
                    df[col + '_std'] = df[col].apply(np.std)
                    df[col + '_min'] = df[col].apply(np.min)
                    df[col + '_max'] = df[col].apply(np.max)
                    
                    df.drop(columns=[col], inplace=True)
            except Exception as e:
                print(f"Skipping column {col} due to error: {e}")
    
    print(f"Summarized columns: {array_cols}")
    return df

# Summarization function
combined_df_summarized = summarize_array_features(combined_df)

# Drop unnecessary columns
combined_df_summarized.drop(columns=['file_name'], inplace=True, errors='ignore')

print(combined_df_summarized.dtypes)

Summarized columns: []
mel_spectrogram       float64
spectrogram           float64
mfccs                 float64
delta_mfccs           float64
delta2_mfccs          float64
zero_crossing_rate    float64
chroma_features       float64
cqt                   float64
log_spectrogram       float64
label                 float64
dtype: object


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

real_csv_path = 'real_audio_features.csv'
fake_csv_path = 'fake_audio_features.csv'

real_df = pd.read_csv(real_csv_path)
fake_df = pd.read_csv(fake_csv_path)

combined_df = pd.concat([real_df, fake_df], ignore_index=True)

# Shuffle the dataset
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract features and labels
X = combined_df.drop(columns=['filename', 'label'])
y = combined_df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # To ensure balanced splits
)

print(f"Training set label distribution:\n{y_train.value_counts()}")
print(f"Test set label distribution:\n{y_test.value_counts()}")

Training set label distribution:
label
0    936
1    883
Name: count, dtype: int64
Test set label distribution:
label
0    234
1    221
Name: count, dtype: int64


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")
X_train = X_train.drop(columns=['filename'], errors='ignore')  # Assuming filename was included
X_test = X_test.drop(columns=['filename'], errors='ignore')    # Same for test data
# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(eval_metric='logloss')

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Detailed classification report
print(classification_report(y_test, y_pred))
# Optionally, save the model for later use
xgb_model.save_model('xgb_model.model')

In [16]:
from sklearn.metrics import roc_auc_score

# Prediction probabilities
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

ROC-AUC Score: 0.9987817612252001


In [20]:
import numpy as np
from sklearn.metrics import roc_curve

# FPR, TPR and thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# FRR = 1 - TPR
frr = 1 - tpr

# Threshold where FAR = FRR
eer_threshold = thresholds[np.nanargmin(np.absolute((fpr - frr)))]

eer = fpr[np.nanargmin(np.absolute((fpr - frr)))]
print(f"Equal Error Rate (EER): {eer}")

Equal Error Rate (EER): 0.01282051282051282
